<a href="https://colab.research.google.com/github/Abdul-AI-creator/Abdul-AI-creator/blob/main/Pytorch_intermediate(bidirectional_recurrent_neural_network).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Important Libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# device Comfiguration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

Hyperparameters

In [3]:
seq_length = 28
n_classes = 10
input_size =28
hidden_size=128
n_layers = 2
n_epochs = 2
batch_size = 100
learning_rate = 0.003

Dataset

In [4]:
train_ds = torchvision.datasets.MNIST(root = './data', train=True , transform = transforms.ToTensor() ,download=True)
train_ds

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 131MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 42.6MB/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.95MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [5]:
test_ds = torchvision.datasets.MNIST(root ='./data' ,train=False , transform = transforms.ToTensor())
test_ds

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()

DataLoaders

In [6]:
train_loader = torch.utils.data.DataLoader(dataset =train_ds ,batch_size=batch_size ,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset =test_ds , batch_size = batch_size ,shuffle =False )

Bidirectional Recurrent Neural Network(many to one)

In [15]:
class BiRNN(nn.Module):
  def __init__(self ,input_size ,hidden_size ,n_layers ,n_classes):
    super(BiRNN ,self).__init__()
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    self.lstm = nn.LSTM(input_size ,hidden_size ,n_layers ,batch_first=True ,bidirectional=True) #  long short-term memory (LSTM)
    self.fc = nn.Linear(hidden_size*2 , n_classes)    # 2 for Bidirectional

  def forward(self ,x):
          # set initial state
    h0 = torch.zeros(self.n_layers*2 ,x.size(0) ,self.hidden_size).to(device)
    c0 = torch.zeros(self.n_layers*2 ,x.size(0) ,self.hidden_size).to(device)
    # Forward Propagate LSTM
    out, _ =self.lstm(x, (h0 ,c0))      # out:  tensor of shape (batch_size ,seq_length ,hidden_size*2)
    # decode the hidden state of the last time step
    out = self.fc(out[: , -1 , :])
    return out

model =BiRNN(input_size ,hidden_size ,n_layers , n_classes).to(device)
model

BiRNN(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=10, bias=True)
)

**Loss and Optimizer**

In [16]:
criterion =nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters() ,lr = learning_rate)

**Train the model**

In [17]:
total_step =len(train_loader)
for epoch in range(n_epochs):
  for i ,(images ,labels) in enumerate(train_loader):
    images = images.reshape(-1 ,seq_length ,input_size).to(device)
    labels = labels.to(device)
    # forward pass
    outputs = model(images)
    loss = criterion(outputs ,labels)

    # backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 ==0:
      print('Epoch [{}/{}], Step [{}/{}] ,Loss: {:.4f}'.format(epoch+1 ,n_epochs ,i +1, total_step ,loss.item()))

Epoch [1/2], Step [100/600] ,Loss: 0.6722
Epoch [1/2], Step [200/600] ,Loss: 0.4898
Epoch [1/2], Step [300/600] ,Loss: 0.2111
Epoch [1/2], Step [400/600] ,Loss: 0.2260
Epoch [1/2], Step [500/600] ,Loss: 0.2155
Epoch [1/2], Step [600/600] ,Loss: 0.0977
Epoch [2/2], Step [100/600] ,Loss: 0.1057
Epoch [2/2], Step [200/600] ,Loss: 0.0445
Epoch [2/2], Step [300/600] ,Loss: 0.0940
Epoch [2/2], Step [400/600] ,Loss: 0.1294
Epoch [2/2], Step [500/600] ,Loss: 0.1141
Epoch [2/2], Step [600/600] ,Loss: 0.0334


**Test the Model**

In [19]:
with torch.no_grad():
  correct =0
  total = 0
  for images , labels in test_loader:
    images = images.reshape(-1 , seq_length ,input_size).to(device)
    labels = labels.to(device)
    outputs =model(images)
    _, predicted =torch.max(outputs.data ,1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print('Test Accuracy on model of 10000 test images: {} %'.format(100 * correct/total))
torch.save(model.state_dict() ,'BiRNN_model.csv')

Test Accuracy on model of 10000 test images: 97.71 %
